### Семинар 9. Алгоритмы и лямбды

<br />

##### Откуда ноги растут - из функторов

Во времена до С++11:

In [ ]:
bool is_underage(const Person& person)
{
    return person.age < 18;
}

std::vector<Person> people = { ... };

auto it = std::find_if(people.begin(), people.end(), is_underage);

Либо через функтор:

In [ ]:
struct IsUnderageChecker
{
    bool operator()(const Person& person) const
    {
        return person.age < 18;
    }    
};

std::vector<Person> people = { ... };
        
auto it = std::find_if(people.begin(), people.end(), IsUnderageChecker());

Функтор - класс с определённым `operator()`. Объекты такого класса можно "вызывать":

Пример:

In [ ]:
Person ilya{"Ilya Muromec", 32};

IsUnderageChecker is_underage_checker;

if (is_underage_checker(ilya))
    std::cout << "Ilya is old";
else
    std::cout << "Ilya is young";

Конкретно для этого примера (с некоторыми оговорками) не важно, свободная функция `is_underage` или функтор `IsUnderageChecker`, но иногда может понадобится и класс.

**Упражение**: Приведите пример. Подсказка: в чём большое отличиие функции от класса?

Вариант:

In [ ]:
class АgeLessChecker
{
public:
    explicit АgeLessChecker(int bound)
        : bound_(bound)
    {}
    
    bool operator()(const Person& person) const
    {
        return person.age < bound_;
    }
    
private:
    int bound_;
};


Person ilya{"Ilya Muromec", 32};

АgeLessChecker ilya_maturity_checker{33};

if (ilya_maturity_checker(ilya))
    std::cout << "time to fight!";

Вариант:

In [ ]:
class NameInSetChecker
{
public:
    explicit NameInSetChecker(std::set<std::string> allowed_names)
        : allowed_names_(std::move(allowed_names))
    {
    }
    
    bool operator()(const Person& person) const
    {
        return allowed_names_.count(person.name) > 0;
    }
    
private:
    std::set<std::string> allowed_names_;
};


std::vector<Person> people = { ... };

auto it = std::find_if(people.begin(), people.end(),
                       NameInSetChecker{{"Balin", "Dvalin", "Gloin"}});

Более интересный пример:

In [ ]:
struct HTMLPrinter
{
    void operator()(int x) const { std::cout << "<p> int value = " << x << " </p>"; }
    void operator()(double x) const { std::cout << "<p> dlb value = " << x << " </p>"; }
};

HTMLPrinter printer;
printer(5);    // # <p> int value = 5 </p>
printer(5.0);  // # <p> dbl value = 5.000000 </p>

[Правила работы с функторами](https://cpp.com.ru/meyers/ch6.html). Основное:
* Классы функторов для алгоритмов передаются по значению, а значит их следует проектировать копируемыми
* Желательно, чтобы функторы для алгоритмов не изменяли своё состояние (нет гарантий, что функтор не будет раскопирован))

<br />

*Функторы - гибкий и богатый инструмент, за исключением того, что требуют большого объёма кода для реализации. Лямбду можно рассматривать как сокращённую запись функтора, который компилятор сгенерирует неявно.*

(утверждение может быть поднатянуто с точки зрения оптимизаций, применяемых компилятором к лямбдам и функторам, но для понимания лямбд вполне подойдёт)

<br />

##### Контекст, захват и передача параметров. Dangling references, правила

Лямбда без состояния:

In [ ]:
auto it = std::find_if(people.begin(), people.end(),
                       [](const Person& p){ return p.age < 18; });

Когда нужно запомнить состояние (сделать поле данных у функтора):

In [ ]:
auto find_first_younger(const std::vector<Person>& people, int age)
{
    return std::find_if(people.begin(), people.end(),
                        [](const Person& p){ return p.age < age; });    
}

// # Не скомпилируется, т.к. компилятор не знает, каким образом
// # хранить age в лямбде. Ему нужно это явно указать.

Захватывать данные в лямбду можно двумя способами:
1. с копированием значения
    * можно представлять как отдельное поле типа `T` в функторе
2. по ссылке
    * можно представлять  как отдельное поле типа `T*` или `T&` в функторе

In [ ]:
int age = 33;
auto too_young = [age](const Person& p){ return p.age < age; }; 
age = 18;
// # age будет скопирован по значению
// # в поиске используется значение 33

std::find_if(people.begin(), people.end(), too_young);

In [ ]:
int age = 33;
auto too_young = [&age](const Person& p){ return p.age < age; }; 
age = 18;
// # age будет использован по ссылке
// # в поиске используется значение 18

std::find_if(people.begin(), people.end(), too_young);

Можно указывать способ захвата "по умолчанию". Тогда все внешние объекты, упоминаемые внутри лямбды, будут захвачены соответственно:

* всё захватывать по значению:

```
int age = 33;
auto too_young = [=](const Person& p){ return p.age < age; };
```

* всё захватывать по ссылке:

```
int age = 33;
auto too_young = [&](const Person& p){ return p.age < age; };
```

Примеры:

In [ ]:
// # объект int очень дешёв для копирования,
// # а разыменовывать по ссылке его дольше,
// # поэтому захватим по значению

int age = read_age_limit();

auto it = std::find_if(people.begin(), people.end(),
                       [age](const Person& p) { return p.age < age; });

In [ ]:
// # объект std::set<std::string> очень дорогой для копирования,
// # лучше потратить немножко на дополнительные jump-ы по ссылкам,
// # поэтому захватим по ссылке

std::set<std::string> allowed_names = ...;
                
auto it = std::find_if(people.begin(), people.end(),
                       [&allowed_names](const Person& p) { return allowed_names.count(p.name) > 0; });

Упражнение: что здесь происходит?

In [ ]:
std::set<std::string> allowed_names = ...;
std::set<std::string>* p_allowed_names = &allowed_names;
                
auto it = std::find_if(people.begin(), people.end(),
                       [p_allowed_names](const Person& p) { return p_allowed_names->count(p.name) > 0; });

<br />

##### Dangling references problem

In [ ]:
auto make_names_checker()
{
    std::set<std::string> allowed_names = ...;                
    return [&allowed_names](const Person& p) { return allowed_names.count(p.name) > 0; };
}


auto names_checker = make_names_checker();
names_checker("Ilya");  // # ooooops

* какая здесь проблема?
* как с ней бороться?
* правила?

<br />

##### Возвращаемый тип

Компилятор автоматически выводит возвращаемый тип, опираясь на тип выражения в `return`. Но можно и указать тип явно:

In [ ]:
auto too_young = [](const Person& p) { return p.age < 18; };

In [ ]:
auto too_young = [](const Person& p) -> bool { return p.age < 18; }

**Упражнение**: в каких случаях это необходимо?

Ответ-пример:

In [ ]:
// # без явного указания типа возвращемого значения код не скопилируется

auto build_full_name = [](const Person& p) -> std::string {

    // # return type is const char *
    if (p.name.empty() && p.surname.empty())
        return "unknown";

    // # return type is std::string
    return p.name + " " + p.surname;
};

In [ ]:
// # без явного указания типа возвращемого значения код не скопилируется

auto find_lowest_point = [](const std::vector<Point>& points) -> std::optional<Point> {

    // # return type is std::nullopt_t
    if (points.empty())
        return std::nullopt;
    
    // # return type is Point
    return *std::min_element(points.begin(), points.end(),
                             [](const Point l, const Point r) { return l.z < r.z; });
};

<br />

##### mutable - лямбды

Использование ключевого слова `mutable` означает, что данные, захваченные копированием значения, лямбда вправе поменять:

In [ ]:
void mutable_lambda_example()
{
    auto fun = [i = 0]() mutable -> bool {
        ++i;
        return i != 4;
    };
    while (fun())
        std::cout << "ho";  // # напечатает hohoho
}

<br />

##### immediately call labmdas

Рассмотрим пример:

In [ ]:
void process_log_file(const std::string& basename)
{
    const std::string filepath = basename + ".log";
    std::ifstream ifs(filepath);
    if (!ifs)
        throw std::runtime_error("filed to open log file on read");

    std::vector<Record> log_records{std::istreambuf_iterator<Record>(ifs),
                                    std::istreambuf_iterator<Record>()};
    std::reverse(log_records.begin(), log_records.end());
    
    
    ... 200+ lines of records processing further
}

Проблемы такой реализации:
* скорее всего `filepath` и `ifs` больше не нужны, но:
    - переменные видны в коде ниже и надо разбираться, нужны они там или нет
    - переменные держат за собой память и открытый дескриптор файла, а эти ресурсы уже можно и освободить
* формирование `log_records` завершено и далее меняться не будет. Его бы как-нибудь пометить `const`, но язык такое не позволяет.

Возможные решения: выделить функцию `read_log_records` или завернуть в immediately call lambda:

In [ ]:
void process_log_file(const std::string& basename)
{
    const auto log_records = [&basename](){
        const std::string filepath = basename + ".log";
        std::ifstream ifs(filepath);
        if (!ifs)
            throw std::runtime_error("filed to open log file on read");

        std::vector<Record> log_records{std::istreambuf_iterator<Record>(ifs),
                                        std::istreambuf_iterator<Record>()};
        std::reverse(log_records.begin(), log_records.end());
        return log_records;
    }();
    
    ... 200+ lines of records processing further
}

Не стоит бояться оверхеда в релизе на вызов immediately call - лямбд. Компиляторы хороши в их оптимизации:

(закинуть код на godbolt.org)

In [ ]:
// # оптимизация для compile-time вычислений
int get_5() noexcept
{
    return [](){ return 5; }();    
}

// # оптимизация для чуть более сложных функций
double f_1(const double x, const double y) noexcept
{
    const double l = x*x +  y*y;
    return x / l + y / l;
}

double f_2(const double x, const double y) noexcept
{
    const double add1 = [=](){
        const double l = x*x + y*y;
        return x / l;
    }();
    const double add2 = [=](){
        const double l = x*x + y*y;
        return y / l;
    }();
    return add1 + add2;
}

Конечно, пример не есть доказательство, но, тем не менее, и gcc, и clang смогли сгенерировать одинаковый код для `f_1` и `f_2`, несмотря на наличие лишних вызовов и доп. вычислений в `f_2`.

На уровне оптимизаций для отладки `-O0` компилятор не может себе позволить инлайнить immediately call лямбды, там код отличается, это нормально.

Поэтому, если performance для `-O0` важен (бывают такие проекты), то в горячих функциях (коих **очень мало**) лучше воздержаться от immediately call.

<br />

##### generic lambda

Ничто не мешает сделать `operator()` шаблонным:

In [ ]:
struct HTMLPrinter
{
    template<typename T>
    void operator()(const T& x) const { std::cout << "<p> value = " << x << " </p>"; }
};

Аналогично можно поступить и для лямбды, заменив тип аргумента словом `auto`

In [ ]:
auto too_young = [](const auto& p) { return p.age < 18; };

generic lambda как и template-метод позволяют вызывать для разных аргументов

In [ ]:
bool f()
{
    auto less = [](const auto& x, const auto& y) { return x < y; };
    return less(3, 5) && less(3.0, 5.0);
}

Пару слов об использовании и цене

<br />

##### `noexcept` lambda

`noexcept` имеет аналогичную семантику как и `noexcept` для обычных функций.

In [ ]:
auto too_young = [](const auto& p) noexcept { return p.age < 18; };

<br />

##### Сохранение лямбда-функции как объекта.

Проблема в записи лямбды в поле класса заключается в том, что тип лямбда-функции создаётся в момент генерации лямбды. Его нельзя выписать явно (иногда можно неявно).

Варианты решения проблемы:
1. Использовать шаблоны
2. Заворачивать лямбды в `std::function`

In [ ]:
class ImagesManager
{
public:
    ImagesManager(std::function<Image(int)> img_searcher)
        : img_searcher_(std::move(img_searcher))
    {}
    
    ...
    
private:
    std::function<Image(int, int)> img_searcher_;
};


// # client code:
auto search_image = [](const int key) -> Image { ... };
ImagesManager manager(search_image);

In [ ]:
template<typename ImgSearcherT>
class ImagesManager
{
public:
    ImagesManager() {}
    
    ...
    
private:
    ImgSearcherT img_searcher_;
};


// # объяснить, почему это вообще работает
// # client code:
auto search_image = [](const int key) -> Image { ... };
ImagesManager<decltype(search_image)> manager;

**Упражнение**: каковы плюсы и минусы обоих из вариантов?